In [ ]:
import os
import requests
import zipfile
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from pprint import pprint as pp

In [ ]:
# Import data from MovieLens

In [ ]:
url = "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

# Create the 'data' directory if it doesn't exist
data_dir = '../data'
os.makedirs(data_dir, exist_ok=True)

# Save the ZIP file inside the 'data' directory
zip_path = os.path.join(data_dir, 'ml-latest-small.zip')

# Get the data
response = requests.get(url, stream=True)

# Save the data
with open(zip_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

# Extract the ZIP file within the 'data' directory
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # zip_ref.extractall()
    for member in zip_ref.namelist():
        target_path = os.path.join(data_dir, member)
        zip_ref.extract(member, path=data_dir)

print(f"Dataset downloaded and extracted to: {data_dir}")

In [ ]:
## Check the links data

In [ ]:
links_df = pd.read_csv(
    '../data/ml-latest-small/links.csv', 
    usecols=['movieId', 'imdbId', 'tmdbId'],
    dtype={'movieId': int, 'imdbId': str, 'tmdbId': str}
)
links_df

In [ ]:
"""
When I initially loaded the imdbId values as ints, some had only 6 digits and were missing
a preceding 0 which caused them to no correspond to the correct IMDB url. When loaded as 
strings, they appear to have the preceding 0s but we need to check this.
"""

# Check for imdbIds of the wrong length.
links_df[links_df.imdbId.apply(lambda x:len(x)!=7)]

In [ ]:
# Check for NaN vals
print(links_df.isna().sum()) 

In [ ]:
''' 
We need to make sure these movieIds are filtered out of the other datasets because we are training
the Matrix Factorization model on the tmdb data (see web-scraping section below) and we need all the models to be working on the same data.
'''

links_df[links_df.isna().tmdbId==True]

In [ ]:
## Filter out movies with missing tmdbIds

In [ ]:
### movies.csv

In [ ]:
movies_missing_tmdbIds = links_df[links_df.isna().tmdbId==True]

movies_df = pd.read_csv('../data/ml-latest-small/movies.csv')
movies_df.head()

In [ ]:
movies_df.shape

In [ ]:
""" These don't look like popular movies anyway. """
movies_df[movies_df.index.isin(movies_missing_tmdbIds.index)]

In [ ]:
""" The eight movies have been removed. """
movies_df = movies_df[~movies_df.index.isin(movies_missing_tmdbIds.index)]
movies_df.shape

In [ ]:
### ratings.csv

In [ ]:
ratings_df = pd.read_csv('../data/ml-latest-small/ratings.csv')
ratings_df.head()

In [ ]:
ratings_df.shape

In [ ]:
"""
Let's just double check how much data we're dropping.
-> No big deal. These 8 movies have only been rated once or twice anyway.
"""
ratings_df[ratings_df.movieId.isin(movies_missing_tmdbIds.movieId)]

In [ ]:
""" Remove the movies. """
ratings_df = ratings_df[~ratings_df.movieId.isin(movies_missing_tmdbIds.movieId)]
ratings_df.shape

In [ ]:
### tags.csv

In [ ]:
tags_df = pd.read_csv('../data/ml-latest-small/tags.csv')
tags_df.head()

In [ ]:
tags_df.shape

In [ ]:
""" Remove the movies. """
tags_df = tags_df[~tags_df.movieId.isin(movies_missing_tmdbIds.movieId)]
tags_df.shape

In [ ]:
### links.csv

In [ ]:
print(f"Num links before: {links_df.shape[0]}")
links_df = links_df[~links_df.movieId.isin(movies_missing_tmdbIds.movieId)]
print(f"Num links after: {links_df.shape[0]}")

In [ ]:
## Overwrite the CSVs

In [ ]:
movies_df.to_csv('../data/ml-latest-small/movies.csv', index=False)
ratings_df.to_csv('../data/ml-latest-small/ratings.csv', index=False)
tags_df.to_csv('../data/ml-latest-small/tags.csv', index=False)
links_df.to_csv('../data/ml-latest-small/links.csv', index=False)

In [ ]:
## Reload the data and check the size

In [ ]:
movies_df = pd.read_csv('../data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('../data/ml-latest-small/ratings.csv')
tags_df = pd.read_csv('../data/ml-latest-small/tags.csv')
links_df = pd.read_csv(
    '../data/ml-latest-small/links.csv', 
    usecols=['movieId', 'imdbId', 'tmdbId'],
    dtype={'movieId': int, 'imdbId': str, 'tmdbId': str}
)

print(f"movies_df shape: {movies_df.shape}")
print(f"ratings_df shape: {ratings_df.shape}")
print(f"tags_df shape: {tags_df.shape}")
print(f"links_df shape: {links_df.shape}")

links_df.head()